In [1]:
# importeer de benodigde bibliotheken, dan gaat het later sneller 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from kaggle.api.kaggle_api_extended import KaggleApi
import os
import zipfile
from pathlib import Path

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor
from tqdm import tqdm_notebook, tqdm

import random

# interactieve modus inschakelen en output verbreden
%matplotlib inline
pd.set_option('display.width', 800)

# remove future warnings
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings("ignore")

# We egbruiken steeds deze seed voor reproduceerbaarheid
seed = 42

# path settings
p = Path()
download_path = p / 'data'
output_path = p / 'output'

# Kaggle settings
api = KaggleApi()
api.authenticate()  

In [2]:
# Overzicht files en folders
def print_files_folders(path=p):
    print(path)
    for dirname, _, filenames in os.walk(download_path):
        for filename in filenames:
            print(os.path.join(dirname, filename))
            if 'train.csv' in filename:
                print("hier:",os.path.join(dirname, filename))
                train_set = os.path.join(dirname, filename)
                break

def zoek_train_set(path):
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            if 'train' in filename: 
                train_set = os.path.join(dirname, filename)
                return train_set
def zoek_test_set(path):
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            if 'test' in filename: 
                train_set = os.path.join(dirname, filename)
                return train_set



In [3]:
# path settings
from pathlib import Path
p = Path()
download_path = p / 'data'
output_path = p / 'output'
images_path = p / 'images'

# Create the output directory if it does not exist
if not os.path.exists(output_path):
		os.makedirs(output_path)
# Create the images directory if it does not exist
if not images_path.exists():
	images_path.mkdir(parents=True, exist_ok=True)

# Functie om de output van de gemaakte plots te bewaren
def save_fig(fig_name, tight_layout=True, fig_extension="png", resolution=300):
    path = images_path / f"{fig_name}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Lees het bestand in
train_set = 'data/train.csv'
train_raw = pd.read_csv(train_set, index_col='ID') # hier gebruiken we de ID-kolom als index !!
test = pd.read_csv('data/test.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')
# Vanaf hier werken we verder met 'data' als de trainingset
# train_test_split wordt dan X_train en X_valid
data = train_raw.copy()

Data cleaning en preprocessing 

# Profiling
Om een eerste indruk te krijgen over de dataset, maken we gebruik van een profiler.   
In het geëxporteerde bestand /output/profiler.html krijg je hierna een mooi overzicht van alle data waaruit deze dataset is opgebouwd.   
 

In [ ]:
##########################
# PROFILER HTML DOCUMENT #
##########################

from ydata_profiling import ProfileReport
import os
import webbrowser

# controleer of er al een rapport is
profiler_file = 'profiler_3_PL.html'
profiler_path = os.path.abspath(os.path.join(output_path, profiler_file))
if os.path.exists(profiler_path):
	print(f"profiler betsaat al. Openen...")
else:
	# Generate the profiling report, kies een goede titel
	profile = ProfileReport(data, title="Baseline London-house-price Report", explorative=True) # explorative=True om ook de correlaties te zien



	# Save the report as an HTML file
	profile.to_file(os.path.join(output_path,profiler_file))

	# Pad naar je bestand
	profiler_path = os.path.abspath(os.path.join(output_path, profiler_file))
	print(f"profiler gemaakt: {profiler_path}")

# Open het bestand in de standaardbrowser
webbrowser.open(f"file://{profiler_path}")




In de Profiler kan je eigenlijk alles al zien.   
Hieronder enkele functies om hier een output te krijgen    


In [ ]:
# we 'commenten' alles om bij een Run All niet steeds de outputs te moeten maken
""""
print(f"Eerste 5 rijen")
print(f"--------------")
print(data.head())
print(f"Info over de kolommen")
print(f"-----------------------")
print(data.info())
print(f"statistische info over de numerieke kolommen")
print("------------------------")
print(f"{data.describe(include='all')}")         #
print(f"aantal rijen en kolommen)
print(f"------------------------")
print(f"{data.shape}")              # 
print(f"kolomnamen")
print(f"------------------------")
print(f"{data.shape}")              # 
print(f"datatype van de kolommen")
print(f"------------------------")
print(f""{data.columns}")            # 
print(f"datatype van de kolommen")
print(f"------------------------")
print(f"{data.dtypes}")             # 
print(f"aantal missende waarden per kolom")
print(f"------------------------")
print(f"{data.isnull().sum()}")     # 
print(f"aantal unieke waarden per kolom")
print(f"------------------------")
print(f"{data.nunique()}")          # 

# Nog een methode is dfsummary
from summarytools import dfSummary
dfSummary(data) # geeft een mooi overzicht van de data


"""
None

### Vooraleer te starten, eerst wat opschonen

In [4]:
# Missende waarden invullen in test- en train-data en kilommen met te veel lege waarden droppen

# Functie om NaN-waarden te behandelen
# Lijst om kolommen met >50% missende waarden op te slaan
cols_to_drop = []

def fill_missing_values(df):
    for col, missing_count in df.isnull().sum().items():
        missing_ratio = missing_count / len(df)
        
        if 0 < missing_ratio <= 0.5:
            if df[col].dtype == object:  # Categorische kolommen
                df[col] = df[col].fillna(df[col].mode()[0])
            else:  # Numerieke kolommen
                df[col] = df[col].fillna(df[col].median())
        
        elif missing_ratio > 0.5: # meer dan de helft van de waarden ontbreken
            cols_to_drop.append(col)

# Kolommen met te veel missende waarden verwijderen
data = data.drop(columns=cols_to_drop)

del cols_to_drop  # Opschoonactie


# alleen uitvoeren indien nog lege waarden
if data.isnull().sum().sum() > 0:
    fill_missing_values(test)
    fill_missing_values(data)
    print(f"De lege waarden werden opgevuld en de kolommen met te veel missende waarden zijn verwijderd")
else:
    print("Er zijn geen missende waarden")

# Onnodige cols schrappen
# alleen uitvoeren indien de kolommen nog aanwezig zijn
if 'ID' in data.columns:
    data = data.drop(columns=['ID', 'country'])
    print(f"De kolommen 'ID' en 'country' zijn verwijderd")



De lege waarden werden opgevuld en de kolommen met te veel missende waarden zijn verwijderd


Verwijderen van enkele kolommen

In [5]:
# indien geen kolom postcode aanwezig is, dan deze toevoegen
if 'postcode' not in data.columns and 'fullAddress' in data.columns:
    # Extracteer postcode uit adres (bijvoorbeeld 'SE5 8AB' uit 'Flat 6, 7 De Crespigny Park, London, SE5 8AB')
    data['postcode'] = data['fullAddress'].str.extract(r'(\w{1,2}\d{1,2} \d{1,2}[A-Z]{1,2})', expand=False)
# Verwijder de volledige adreskolom
if 'fullAddress' in data.columns:
    data.drop(columns='fullAddress', inplace=True)
    print(f"Full address is verwijderd en de postcode is toegevoegd.")
else:
    print("De fullAddress kolom reeds verwijderd.")

# slechts één waarde voor country, dus ook weg ermee
if 'country' in data.columns: 
    data.drop(columns='country', inplace=True)
    print(f"Country is verwijderd")


Full address is verwijderd en de postcode is toegevoegd.
Country is verwijderd


Deze is belangrijk.   
Gezien de spreiding van de 'Price' kies ik ervoor om een Log-transformatie toe te passen op deze waarden 

In [ ]:
# Target-transformatie (Log-transformatie op `price`) indien niet eerder gebeurd
if data['price'].max() > 100:
    data['price'] = np.log1p(data[' price'])  # log(1 + price) om log(0) te voorkomen, maar hier geen 0-waarden
    print(f"Log-transformatie op 'price' uitgevoerd")
else:
    print("Log-transformatie op 'price' werd reeds uitgevoerd")


Log-transformatie op 'price' uitgevoerd


# Hier gaat het gebeuren

Stap 1: Definieer de preprocessing stappen
Label Encoding voor categorische features en StandardScaler voor numerieke features gebruiken.

Stap 2: Bouw de pipeline
We maken de pipeline voor elke modelvariant en voegen de preprocessing in de pipeline in.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import randint
import xgboost as xgb
from catboost import CatBoostRegressor


# 1. Target-transformatie (Log-transformatie op `price`)
if data['price'].max() > 100:
    data['price'] = np.log1p(data[' price'])  # log(1 + price) om log(0) te voorkomen, maar hier geen 0-waarden
    print(f"Log-transformatie op 'price' uitgevoerd.\n")
else:
    print("Log-transformatie op 'price' werd reeds uitgevoerd.\n")

# 2. Splitsen in features (X) en target (y)
X = data.drop(columns=['price'])
y = data['price']

# 3. Train-test split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=seed)

# 4. Pipeline setup
num_features = ['bathrooms', 'bedrooms', 'floorAreaSqM', 'livingRooms']
cat_features = ['tenure', 'propertyType', 'currentEnergyRating']

# Preprocessor
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_features),
    ('cat', OrdinalEncoder(), cat_features),
])

# Definieer de pipelines voor verschillende modellen
pipelines = {
    'LinearRegression': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())
    ]),
    'RandomForest': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(random_state=seed))
    ]),
    'GradientBoosting': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', GradientBoostingRegressor(random_state=seed))
    ]),
    'CatBoost': Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', CatBoostRegressor(verbose=0, random_state=seed))
    ])
}

# Hyperparameter grids voor elk model
param_grids = {
    'LinearRegression': {},  # Geen hyperparameter tuning nodig
    'RandomForest': {
        'regressor__n_estimators': [50, 100, 200],
        'regressor__max_depth': [5, 10, 20],
        'regressor__min_samples_split': [2, 5, 10]
    },
    'GradientBoosting': {
        'regressor__n_estimators': [50, 100, 200],
        'regressor__learning_rate': [0.01, 0.1, 0.2],
        'regressor__max_depth': [3, 5, 10]
    },
    'CatBoost': {
        'regressor__iterations': [100, 200, 500],  
        'regressor__learning_rate': [0.01, 0.05, 0.1],  
        'regressor__depth': [4, 6, 10]
}
}

# Train modellen met hyperparameter tuning
best_models = {}

for name, model in pipelines.items():
    print(f"\n🔍 Optimalisatie van {name} met RandomizedSearchCV...")

    if param_grids[name]:  # Alleen tunen als er parameters zijn
        search = RandomizedSearchCV(model, param_distributions=param_grids[name],
                                    n_iter=10, cv=3, scoring='r2',
                                    n_jobs=-1, verbose=2, random_state=seed)
        
        search.fit(X_train, y_train)
        best_models[name] = search.best_estimator_
        print(f"✅ Beste parameters voor {name}: {search.best_params_}")
    else:
        model.fit(X_train, y_train)
        best_models[name] = model
        print(f"✅ {name} is getraind zonder hyperparameter tuning.")

print("\n🎯 Hyperparameter tuning voltooid voor alle modellen!")

# --- 🔹 Evaluatie van alle modellen 🔹 ---
evaluation_results = {}

for name, model in best_models.items():
    print(f"\n📊 Evaluatie van {name}...")

    y_pred = model.predict(X_valid)

    mae = mean_absolute_error(y_valid, y_pred)
    mse = mean_squared_error(y_valid, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_valid, y_pred)

    evaluation_results[name] = {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R²': r2}

    print(f"✅ {name} prestaties:")
    print(f"   MAE: {mae:.4f}")
    print(f"   MSE: {mse:.4f}")
    print(f"   RMSE: {rmse:.4f}")
    print(f"   R²: {r2:.4f}")

# --- 📌 Overzicht van alle resultaten ---
print("\n📌 Modelvergelijking:")
for model, scores in evaluation_results.items():
    print(f"{model}: R²={scores['R²']:.4f}, RMSE={scores['RMSE']:.4f}, MAE={scores['MAE']:.4f}")

# Zet de evaluatieresultaten om naar een pandas DataFrame
df_results = pd.DataFrame.from_dict(evaluation_results, orient='index')

# Opslaan als CSV
df_results.to_csv("output/model_evaluatie_resultaten.csv", index=True)

print("\n📂 Resultaten opgeslagen als 'model_evaluatie_resultaten.csv'")
print(df_results)

import seaborn as sns
import matplotlib.pyplot as plt

# Sorteer modellen op R²-score
df_results_sorted = df_results.sort_values(by='R²', ascending=False)

# Staafdiagram maken
plt.figure(figsize=(8, 5))
sns.barplot(x=df_results_sorted.index, y=df_results_sorted['R²'], palette='viridis')

# Labels en titel
plt.ylabel("R² Score")
plt.xlabel("Model")
plt.title("📊 Modelvergelijking: R²-scores")
plt.ylim(0, 1)  # R² varieert tussen 0 en 1
plt.xticks(rotation=30)  # Kantel labels voor leesbaarheid

# Toon de grafiek
save_fig("model_vergelijking R²-scores")
plt.show()



Log-transformatie op 'price' werd reeds uitgevoerd.


🔍 Optimalisatie van LinearRegression met RandomizedSearchCV...
✅ LinearRegression is getraind zonder hyperparameter tuning.

🔍 Optimalisatie van RandomForest met RandomizedSearchCV...
Fitting 3 folds for each of 10 candidates, totalling 30 fits
✅ Beste parameters voor RandomForest: {'regressor__n_estimators': 200, 'regressor__min_samples_split': 2, 'regressor__max_depth': 10}

🔍 Optimalisatie van GradientBoosting met RandomizedSearchCV...
Fitting 3 folds for each of 10 candidates, totalling 30 fits
✅ Beste parameters voor GradientBoosting: {'regressor__n_estimators': 100, 'regressor__max_depth': 5, 'regressor__learning_rate': 0.1}

🔍 Optimalisatie van CatBoost met RandomizedSearchCV...
Fitting 3 folds for each of 10 candidates, totalling 30 fits
✅ Beste parameters voor CatBoost: {'regressor__learning_rate': 0.1, 'regressor__iterations': 500, 'regressor__depth': 6}

🎯 Hyperparameter tuning voltooid voor alle modellen!

📊 Evaluatie va

OSError: Cannot save file into a non-existent directory: 'outpunt'

In [ ]:
y_pred_log = best_pipeline.predict(test)
y_pred = np.round(np.expm1(y_pred_log),0).astype(int)

In [ ]:
# nodig om het resultaat in te dienen
sub = sample_submission.copy()
sub['price'] = y_pred
sub.to_csv('output/submission_cbr_log.csv', index=False)
sub